In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Get camera capture
cap = cv2.VideoCapture(0)  # 0 for default camera, change if necessary

# Get camera frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a crosshair at the center of the screen
crosshair_color = (0, 255, 0)  # Green color
crosshair_thickness = 2
crosshair_length = 20
crosshair_center = (frame_width // 2, frame_height // 2)

# Threshold for object proximity to the crosshair
threshold = 50

# Define ranges for red and green colors in HSV format
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])
lower_green = np.array([50, 100, 100])
upper_green = np.array([70, 255, 255])

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Threshold the HSV image to get only red and green colors
        red_mask = cv2.inRange(hsv_frame, lower_red, upper_red)
        green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)

        # Combine the masks to detect both red and green colors
        mask = cv2.bitwise_or(red_mask, green_mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize laser centroid
        laser_center = None

        # Check if any contour is found
        if contours:
            # Find the largest contour (assuming it's the laser point)
            largest_contour = max(contours, key=cv2.contourArea)
            # Calculate the centroid of the contour
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                laser_center = (cX, cY)

        # If laser is detected within the specified range, update crosshair position to lock onto it
        if laser_center:
            distance = ((laser_center[0] - crosshair_center[0]) ** 2 + (laser_center[1] - crosshair_center[1]) ** 2) ** 0.5
            if distance < threshold:
                crosshair_center = laser_center

        # Draw crosshair on the frame
        cv2.line(frame, (crosshair_center[0] - crosshair_length, crosshair_center[1]), 
                 (crosshair_center[0] + crosshair_length, crosshair_center[1]), crosshair_color, crosshair_thickness)
        cv2.line(frame, (crosshair_center[0], crosshair_center[1] - crosshair_length), 
                 (crosshair_center[0], crosshair_center[1] + crosshair_length), crosshair_color, crosshair_thickness)

        # Display the frame
        clear_output(wait=True)
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

except KeyboardInterrupt:
    # Release camera capture
    cap.release()
    cv2.destroyAllWindows()


KeyboardInterrupt: 